In [128]:
import sys
import os

from yahoo_oauth import OAuth2
import yahoo_fantasy_api as yfa 

# Get the absolute path to the 'src' directory
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))

# Add the 'src' directory to sys.path
sys.path.append(src_path)

from db_utils import fantasy_db

In [129]:
oauth = OAuth2(None, None, from_file='../src/oauth2.json')

[2024-10-25 11:55:59,263 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2024-10-25 11:55:59,267 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 22.437687397003174
[2024-10-25 11:55:59,268 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [130]:
gm = yfa.Game(oauth, 'nba')

In [131]:
# https://yahoo-fantasy-api.readthedocs.io/en/latest/yahoo_fantasy_api.html#the-league-class

lg = yfa.League(oauth, '454.l.222542')

In [5]:
## Get Entries
db_conn = fantasy_db()

In [21]:
db_conn.cursor.execute("SELECT selected_player_id, salary FROM entries;")

selected_players =db_conn.cursor.fetchall()

In [22]:
# from DFS entry
selected_players

[(6044, 45),
 (6569, 27),
 (5357, 25),
 (6698, 11),
 (6171, 20),
 (5693, 15),
 (6558, 42),
 (5643, 13)]

In [23]:
lg.player_stats(6044, req_type = 'average_season', season = 2023)

[{'player_id': 6044,
  'name': 'Jalen Brunson',
  'position_type': 'P',
  'PTS': 28.7,
  'REB': 3.6,
  'AST': 6.7,
  'ST': 0.9,
  'BLK': 0.2,
  'TO': 2.4,
  'total_points': 43.97}]

In [135]:
# Evaluate 

class evaluate_entries:
    def __init__(self, players:list):
        """
        Args:
            players (list): A list of tuples where (player_id, salary)

        """
        self.selected_players = [i[0] for i in players]

        # Build a profile
        self.players_profile = {}
        
        for i in players:
            self.players_profile[i[0]] = {'salary':i[1], 'projected_fps':0, 'value_score': 0}


        # Scoring        
        self.scoring_criteria = { 'PTS':1, 'REB': 1.2, 'AST':1.5, 'ST':3, 'BLK':3, 'TO':-1}
        self.projected_score = 0
        self.average_player_value = 0
        
        #self.df_temp = pd.DataFrame(columns = [])
        

    def calculate_FP(self, player_stats:dict):
        """
        Calculate the fantasy points (FP) for a player based on their statistics and the scoring criteria.

        Args:
            player_stats (dict): A dictionary containing the player's statistics, where keys are the stat names
                                and values are the stat values.

        Returns:
            float: The calculated fantasy points score for the player.
        """
        fps_score = 0
        
        for criteria in self.scoring_criteria.keys():
            fps_score += player_stats[criteria] * self.scoring_criteria[criteria]

        return fps_score

    def value_score(self, player_id, method = 'simple'):
        """
        Assign a 'value score' against statistics and salary. In general, the higher the value, the more stats per salary

        Args:
            method: an approach
                simple = projected_fps/salary

        Returns:
            float: a ratio

        """

        if method == 'simple':
            return self.players_profile[player_id]['projected_fps'] / self.players_profile[player_id]['salary'] 
        

        

    def evaluate_roster(self, **kwargs):

        average_player_value = 0

        for player_id in self.selected_players:

            #fetch stats from API
            player_stats = lg.player_stats(player_id, req_type = 'average_season', season = 2023)[0]  # trying to see if i can parameterize this line.
            fps_score = self.calculate_FP(player_stats)
            self.projected_score += fps_score


            self.players_profile[player_id]['projected_fps'] = fps_score
            self.players_profile[player_id]['value_score'] =   self.value_score(player_id)    
            
            average_player_value += self.players_profile[player_id]['value_score'] 

         
        self.average_player_value = average_player_value / len(self.selected_players)       

        
        

In [136]:
oddone = evaluate_entries(selected_players)

In [137]:
oddone.evaluate_roster()

In [139]:
oddone.average_player_value

1.2229574194324193